In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import visdom

In [3]:
vis = visdom.Visdom()

Setting up a new session...


In [4]:
from preprocessing_utils import process_sat_data, pickle_save, pickle_load
from trajectory import trajectory

In [5]:
data_train = pd.read_csv('../data/train.csv')
data_test = pd.read_csv('../data/track1/test.csv')

In [32]:
train_data = data_train.copy(deep=True)
test_data = data_test.copy(deep=True)

In [33]:
process_sat_data(train_data, test_data, scale=10000)

# Rodrigues Rotation

$ {\displaystyle \mathbf {v} _{\mathrm {rot} }=\mathbf {v} \cos \theta +(\mathbf {k} \times \mathbf {v} )\sin \theta +\mathbf {k} ~(\mathbf {k} \cdot \mathbf {v} )(1-\cos \theta )\,} $

In [34]:
sat_id = 1
ellipse_id = 3
ellipse_data = train_data[train_data['sat_id'] == sat_id].loc[:, ['x_sim', 'y_sim', 'z_sim', 'epoch']]
ellipse_data = ellipse_data.iloc[ellipse_id * 24 : (ellipse_id + 1) * 24, :]
time = ellipse_data['epoch'].values
ellipse_data = ellipse_data.iloc[:, [0,1,2]].values
ellipse_mean = ellipse_data.mean(axis=0)
ellipse_data -= ellipse_mean

In [35]:
ellipse_normal = np.cross(ellipse_data[0]-ellipse_data[1], ellipse_data[0]-ellipse_data[2])
ellipse_normal /= np.linalg.norm(ellipse_normal)
if (ellipse_normal[2] < 0):
    ellipse_normal *= -1

In [36]:
ellipse_normal

array([-0.14191971,  0.46144182,  0.87574554])

In [37]:
z_axis = np.array([0,0,1])

In [38]:
theta = np.arccos(np.dot(ellipse_normal, z_axis))

In [39]:
theta / np.pi * 360

57.733334785425953

In [40]:
rotation_axis = np.cross(ellipse_normal, z_axis)
rotation_axis /= np.linalg.norm(rotation_axis)

In [41]:
rotation_axis

array([ 0.95581533,  0.29396778, -0.        ])

In [42]:
def rodrigues_rotate(vect, axis, theta):
    rotated = vect * np.cos(theta)
    rotated += np.cross(axis, vect) * np.sin(theta)
    rotated += axis * np.dot(z_axis, vect) * (1 - np.cos(theta))
    return rotated

In [43]:
rotated = rodrigues_rotate(ellipse_normal, rotation_axis, theta)

In [44]:
np.linalg.norm(rotated)

1.005902961509878

In [45]:
rotated

array([ 0.10400732,  0.03198819,  1.        ])

In [99]:
transformed_data = np.array([rodrigues_rotate(vect, rotation_axis, theta) for vect in ellipse_data])

In [100]:
np.abs(transformed_data).mean(axis=0)

array([  1.11730685e+00,   1.15571327e+00,   4.36417967e-04])

In [101]:
transformed_data = transformed_data[:,[0,1]]

# 2D Ellipse Fitting

In [261]:
class Ellipse(nn.Module):
    def __init__(self, name, major, minor, cx, cy, theta):
        super(Ellipse, self).__init__()
        self.name = name
        self.major = nn.Parameter(major)
        self.minor = nn.Parameter(minor)
        self.cx = nn.Parameter(cx)
        self.cy = nn.Parameter(cy)
        self.theta = nn.Parameter(theta)
    def forward(self, data):
        x_rot = data[:,0] - self.cx
        y_rot = data[:,1] - self.cy
        x = x_rot * torch.cos(theta) - y_rot * torch.sin(theta)
        y = x_rot * torch.sin(theta) + y_rot * torch.cos(theta)
        res = ((x**2 / (self.major)**2) + (y**2 / (self.minor)**2) - 1) ** 2
        return res

In [281]:
def get_initial_values(data):
    euclidian = []
    pairs = []
    for i in range(23):
        for j in range(i + 1, 24):
            euclidian_distance = np.linalg.norm(data[i] - data[j])
            pairs.append([i, j])
            euclidian.append(euclidian_distance)
    max_index = np.argmax(euclidian) #Index of Longest Chord
    major_axis = data[pairs[max_index][0]] - data[pairs[max_index][1]]
    if (major_axis[1] < 0):
        major_axis *= -1
    major = np.linalg.norm(major_axis) / 2
    center = (data[pairs[max_index][0]] + data[pairs[max_index][1]]) * 0.5
    theta = np.dot(major_axis, [1,0]) / (2 * major)
    return [major, 1, center[0], center[1], -theta]

In [282]:
major, minor, cx, cy, theta = [torch.tensor(val).float() for val in get_initial_values(transformed_data)]

In [283]:
major, minor, cx, cy, theta

(tensor(2.0497), tensor(1.), tensor(0.3737), tensor(-0.2397), tensor(0.7584))

In [284]:
model = Ellipse('E', major, major, cx, cy, theta)

In [285]:
optim = torch.optim.Adam(model.parameters(), 0.001)

In [286]:
data = torch.from_numpy(transformed_data).float()

In [287]:
for itr in range(1000):
    optim.zero_grad()
    pred = model(data)
    loss = torch.sum(pred)
    loss.backward()
    optim.step()
    if (itr%100 == 0):
        print(loss.item())

1.1051629781723022
0.5009495615959167
0.49546492099761963
0.49593639373779297
0.49594855308532715
0.49594825506210327
0.49594825506210327
0.49594825506210327
0.49594825506210327
0.49594825506210327


In [288]:
list(model.parameters())

[Parameter containing:
 tensor(1.9376, requires_grad=True), Parameter containing:
 tensor(1.9376, requires_grad=True), Parameter containing:
 tensor(0.4503, requires_grad=True), Parameter containing:
 tensor(-0.1313, requires_grad=True), Parameter containing:
 tensor(0.7584, requires_grad=True)]

In [186]:
vis.scatter(d, opts={'markersize':4}, win='ellipse')

'ellipse'